# Stack Exc. Data Analysis

In [25]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [48]:
comments=pd.read_csv(r'C:\Users\רפאל רון\Desktop\JB-DATA ANALYST\Data Analyst\Python\Stack Exchange\comments.csv',
                     delimiter = ",",
                     header = 0,
                     names = ["Id", "PostId", "Text", "CreationDate", "UserId"],
                     dtype = {"Id":int,
                     "PostId":int,
                     "Text":str,
                     "UserId":int},
                     index_col = "Id",
                     parse_dates = ["CreationDate"])

In [47]:
posts=pd.read_csv(r'C:\Users\רפאל רון\Desktop\JB-DATA ANALYST\Data Analyst\Python\Stack Exchange\Posts.csv',
                 delimiter = ",",
                 header = 0,
                 names = ["Id", "CreationDate", "Score", "ViewCount", "Body", "OwnerUserId", "LastEditorUserId", "LastEditDate", "LastActivityDate", "Title", "Tags", "ClosedDate"],
                 dtype = {"Id":int,
                 "Score":int,
                 "ViewCount":str,
                 "Body":str,
                 "OwnerUserId":int,
                 "LastEditorUserId":int,
                 "Title":str,
                 "Tags":str},
                 index_col = "Id",
                 parse_dates = ["CreationDate" , "LastEditDate", "LastActivityDate", "ClosedDate"])

In [46]:
users=pd.read_csv(r'C:\Users\רפאל רון\Desktop\JB-DATA ANALYST\Data Analyst\Python\Stack Exchange\Users.csv',
                 delimiter = ",",
                 header = 0,
                 names = ["Id", "Reputation", "CreationDate", "DisplayName", "LastAccessDate", "WebsiteUrl", "Location", "Age", "AboutMe", "Views", "UpVotes", "DownVotes"],
                 dtype = {"Id":int,
                 "Reputation":int,
                 "DisplayName":str,
                 "WebsiteUrl":str,
                 "Location":str,
                 "Age":float, 
                 "AboutMe":str,
                 "Views":int,
                 "UpVotes":int,
                 "DownVotes":int},
                 index_col = "Id",
                 parse_dates = ["CreationDate" , "LastAccessDate"])

In [45]:
votes=pd.read_csv(r'C:\Users\רפאל רון\Desktop\JB-DATA ANALYST\Data Analyst\Python\Stack Exchange\Votes.csv',
                 delimiter = ",",
                 header = 0,
                 names = ["Id", "PostId", "CreationDate", "UserId"],
                 dtype = {"Id":int,"PostId":int,"UserId":int},
                 index_col = "Id",
                 parse_dates = ["CreationDate" ])

#### basic analysis

###### 1.how many post were made every year?

In [54]:
posts.groupby(posts['CreationDate'].dt.year)[['Title']].count()

,Title
CreationDate,
2011,156
2012,1065
2013,1018


#### 2. How many votes were made in each day of the week (Sunday,etc..)


In [57]:
def Name_of_day (day_number):
    DayName= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return DayName[day_number]

In [59]:
votes["DayOfWeek"] = votes["CreationDate"].dt.dayofweek.apply(Name_of_day)

In [60]:
votes.groupby("DayOfWeek")["PostId"].count().to_frame().rename(columns={"PostId":"VotesCount"})\
.sort_values("VotesCount", ascending = False)

,VotesCount
DayOfWeek,
Thursday,869
Friday,735
Wednesday,554
Saturday,376
Tuesday,354
Monday,282
Sunday,189


#### 3.List all comments created on September 19th, 2012

In [62]:
comments["CreationDate"] = pd.to_datetime(comments["CreationDate"])

In [63]:
ezer=(comments["CreationDate"] > "2012-09-19") & (comments["CreationDate"] < "2012-09-20")

In [64]:
comments[ezer]

,PostId,Text,CreationDate,UserId
Id,,,,
3476,887,I had the same thoughts. I'm wondering if ther...,2012-09-19 10:34:12.983,233
3623,890,"hmmm he's described as ""an over-the-top public...",2012-09-19 18:23:37.193,126


##### 4. List all users under the age of 33, living in London

In [65]:
ezer_empty=~users['Location'].isnull()&~users['Age'].isnull()

In [66]:
ezer_age_place=users['Location'].str.contains('London')&(users['Age']<33)

In [68]:
users[ezer_empty&ezer_age_place].head(1)

,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,Age,AboutMe,Views,UpVotes,DownVotes
Id,,,,,,,,,,,
27,101,2011-11-30 18:58:00.620,Noldorin,2011-12-27 00:54:12.993,http://noldorin.com/,"London, United Kingdom",24.0,<p>entrepreneur; graduate in mathematics / the...,0,0,0


### Advanced Analysis

##### 1.Display the number of votes for each post title

In [70]:
join_posts_votes=posts.merge(votes, how = "left", left_index=True, right_on = "PostId")

In [73]:
join_posts_votes.head(1)

,CreationDate_x,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,LastActivityDate,Title,Tags,ClosedDate,PostId,CreationDate_y,UserId,DayOfWeek
3.0,2011-11-30 19:15:54.070,25,1526,<p>Some comedian/actors are given creative fre...,1562,532,2013-09-25 14:15:14.257,2013-09-25 14:15:14.257,To what extent were the actors in Wedding Cras...,<wedding-crashers>,2011-12-24 19:15:54.070,1,2011-11-30,231.0,Wednesday


In [80]:
join_posts_votes.groupby("Title")["Title"].count().to_frame().rename(columns = {"Title":"VotesCount"}).sort_values("VotesCount", ascending = False).head()

,VotesCount
Title,
Have there been other candidates to play Jack Reacher?,27
Where did the speaking-disease come from?,26
Was Disney's The Black Hole the last film to feature an overture?,26
"In The Departed, did the FBI know about the mole in the police department and did they interfere with the investigation?",25
Why does Harry only make use of time travel in The Prisoner of Azkaban?,23


#### 2. Display posts with comments created by users living in the same location as the post creator

In [82]:
comments.rename(columns={"UserId":"CommentUserId"}, inplace=True)
posts_comments = posts.merge(comments, how='inner', left_index=True,right_on='PostId')[["Title","OwnerUserId","CommentUserId"]]

In [84]:
posts_comments_users_p = posts_comments.merge(users, how='inner', left_on='OwnerUserId', right_index=True)\
[["Title","OwnerUserId","CommentUserId", "Location"]].rename(columns={"Location":"PostCreatorLocation"})

In [85]:
posts_comments_users_p_users_c = posts_comments_users_p.merge(users, how='inner', left_on='CommentUserId', right_index=True)\
[["Title","OwnerUserId","CommentUserId", "PostCreatorLocation", "Location"]].rename(columns={"Location":"commentCreatorLocation"})

In [91]:
ezer = posts_comments_users_p_users_c["PostCreatorLocation"]==posts_comments_users_p_users_c["commentCreatorLocation"]

In [92]:
posts_comments_users_p_users_c[ezer]

,Title,OwnerUserId,CommentUserId,PostCreatorLocation,commentCreatorLocation
Id,,,,,
1668,Was Monk's mother also OCD?,52,52,Missouri,Missouri
30,Thriller about a group of kidnappers wearing S...,1312,1312,"Sydney, Australia","Sydney, Australia"
2726,What is the Traitor's Motive?,1590,535,Canada,Canada
3809,Clarification about closing a loop in The Looper,1632,293,Sweden,Sweden
750,Is there more to Gandalf and Galadriel's relat...,835,1221,Sweden,Sweden


#### 3.How many users have never voted ?

In [95]:
users_votes=users.merge(votes, how = "left", left_index = True,right_on='UserId')

In [96]:
users_votes[users_votes["PostId"].isnull()]["DisplayName"].count()

355

##### 4. Display all posts having the highest amount of comments

In [103]:
posts_comments=posts.merge(comments, how = "inner", left_index = True, right_on = ["PostId"], suffixes=('_pst', '_cmn'))\
.groupby("Title")["Title"].count().sort_values(ascending=False).to_frame().rename(columns={"Title":"CommentsCount"})

In [104]:
posts_comments["RankCountCom"]=posts_comments["CommentsCount"].rank(ascending=False, method='dense')

In [105]:
ezer = posts_comments["RankCountCom"]==1


In [106]:
posts_comments[ezer]

,CommentsCount,RankCountCom
Title,,
How do the 'frozen goods' survive the banana stand fire?,5,1.0
"In Iron Man, why is Tony Stark fooled for so long?",5,1.0
How does the budget breakdown for a US TV season compare to a UK TV Season?,5,1.0
Plot summaries for the other two movies related to Moon,5,1.0
Old movie about two pilots and a cheating wife,5,1.0
Who nukes the USA in The Divide?,5,1.0
Why doesn’t Shosanna try to kill Fredrick?,5,1.0
Aliens took over the satellite,5,1.0
"How much money did it take to run ""The Sting""?",5,1.0


##### 6. How many hours in average, it takes to the first comment to be posted after a creation of a new post

In [108]:
min_comments = comments.groupby("PostId")[["CreationDate"]].min()

In [109]:
comment_post = posts.merge(min_comments, how="inner", left_index=True,right_index=True, suffixes=('_pst', '_cmn'))\
[["Title","CreationDate_pst","CreationDate_cmn"]]

In [150]:
comment_post["time_diff"]= (comment_post["CreationDate_cmn"]-comment_post["CreationDate_pst"]) / np.timedelta64(1, 'h')
np.round(comment_post["time_diff"].mean())


7.0

###### 7. Whats the most common post tag?


In [170]:
l= (posts['Tags'].str.split('><'))
new = []
for sublist in l:
    for item in sublist:
        new.append(item)
ezer = pd.Series(new).str.replace("<","").str.replace(">","").to_frame()
ezer.columns = ['Tag']
ezer.groupby('Tag')['Tag'].count().sort_values(ascending=False).to_frame().head(1)

,Tag
Tag,
plot-explanation,665


##### 8. Create a pivot table displaying how many posts were created for each year (Y axis) and each month (X axis) 

In [171]:
posts["Year_Create"]=posts["CreationDate"].dt.year
posts["Month_Create"]=posts["CreationDate"].dt.month

In [172]:
pvt_posts=posts.pivot_table(index="Year_Create",columns="Month_Create",values="Title",aggfunc="count")

In [174]:
pvt_posts.head()

Month_Create,1,2,3,4,5,6,7,8,9,10,11,12
Year_Create,,,,,,,,,,,,
2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,15.0,140.0
2012,92.0,72.0,62.0,88.0,93.0,105.0,105.0,81.0,60.0,75.0,77.0,155.0
2013,129.0,99.0,107.0,92.0,130.0,120.0,134.0,143.0,64.0,NaN,NaN,NaN
